In [3]:
#
import base58
import binascii
import ecdsa
import hashlib
import json
import os
import subprocess
import time

def dSHA256(data):
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

def hash160(s):
    '''sha256 followed by ripemd160'''
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def privkey_to_pubkey(privkey):
    signing_key = ecdsa.SigningKey.from_string(privkey, curve=ecdsa.SECP256k1) # Don't forget to specify the curve
    verifying_key = signing_key.get_verifying_key()

    # Use this code block if the address you gave corresponds to the compressed public key
    x_cor = bytes.fromhex(verifying_key.to_string().hex())[:32] # The first 32 bytes are the x coordinate
    y_cor = bytes.fromhex(verifying_key.to_string().hex())[32:] # The last 32 bytes are the y coordinate
    if int.from_bytes(y_cor, byteorder="big", signed=True) % 2 == 0: # We need to turn the y_cor into a number.
        public_key = bytes.fromhex("02" + x_cor.hex())
    else:
        public_key = bytes.fromhex("03" + x_cor.hex())
    return public_key

In [4]:
# Functions related to generating bitcoin addresses

def encode_base58(s):
    BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    count = 0
    for c in s:  
        if c == 0:
            count += 1
        else:
            break
    num = int.from_bytes(s, 'big')
    prefix = '1' * count
    result = ''
    while num > 0:  
        num, mod = divmod(num, 58)
        result = BASE58_ALPHABET[mod] + result
    return prefix + result

def encode_base58_checksum(b):
    return encode_base58(b + dSHA256(b)[:4])

# https://en.bitcoin.it/wiki/Wallet_import_format
def privkey_to_wif(privkey, compressed_pubkey, testnet):
    if testnet:
        prefix = b"\xEF"
    else:
        prefix = b"\x80"
    
    # if the privkey will correspond to a compressed public key
    if compressed_pubkey: 
        extended = prefix + privkey + b"\x01"
    else:
        extended = prefix + privkey 
        
    extendedchecksum = extended + dSHA256(extended)[:4]
    wif = encode_base58(extendedchecksum)
    
    return wif

# https://learnmeabitcoin.com/guide/wif
def wif_to_privkey(private_key_WIF):
    private_key_full = base58.b58decode(private_key_WIF)
    
    # If the WIF encoding includes the optional "01" byte for compressed privKey,
    # do not include it in the final output.
    if len(private_key_full) == 38:
        private_key = private_key_full[1:-5] 
        print("compressed pubkey")
    else:
        private_key = private_key_full[1:-4] 
        print("not compressed pubkey")
    return private_key


def pk_to_p2pkh(compressed, testnet):
    '''Returns the address string'''
    pk_hash = hash160(compressed)
    if testnet:
        prefix = b'\x6f'
    else:
        prefix = b'\x00'
    return encode_base58_checksum(prefix + pk_hash)

def pk_to_p2sh_p2wpkh(compressed, testnet):
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    rs_hash = hash160(redeemScript)
    if testnet:
        prefix = b"\xc4"
    else:
        prefix = b"\x05"
    return encode_base58_checksum(prefix + rs_hash)

def wif_to_addresses(wif):
#     wif = "cUy9rC6wteKizfu1fgP2abKUWTkJxjqKp2fba91FkU332CFHo6ix"
    privkey = wif_to_privkey(wif)
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("WIF Private key: " + wif)
    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)
    
def privkey_to_addresses(privkey):
    # privkey = bytes.fromhex("AF933A6C602069F1CBC85990DF087714D7E86DF0D0E48398B7D8953E1F03534A")
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)

# Dual funded escrow

Generate all the priv/public keys

In [20]:
# Generate example priv/pubkeys
cust_funding_privkey_hex = "1111111111111111111111111111111100000000000000000000000000000000"
cust_funding_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_funding_privkey_hex)).hex()

cust_change_privkey_hex = "1111111111111111111111111111111111111111111111111111111111111111"
cust_change_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_change_privkey_hex)).hex()

merch_funding_privkey_hex = "5611111111111111111111111111111100000000000000000000000000000000"
merch_funding_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_funding_privkey_hex)).hex()

merch_change_privkey_hex = "7511111111111111111111111111111111111111111111111111111111111111"
merch_change_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_change_privkey_hex)).hex()

merch_privkey_hex = "3911111111111111111111111111111111111111111111111111111111111111"
merch_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_privkey_hex)).hex()

cust_privkey_hex = "7911111111111111111111111111111111111111111111111111111111111111"
cust_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_privkey_hex)).hex()

# cust_payout_privkey_hex = "7711111111111111111111111111111111111111111111111111111111111111"
# cust_payout_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_payout_privkey_hex)).hex()

merch_payout_privkey_hex = "3711111111111111111111111111111111111111111111111111111111111111"
merch_payout_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_payout_privkey_hex)).hex()

# merch_disp_privkey_hex = "3111111111111111111111111111111111111111111111111111111111111111"
# merch_disp_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_disp_privkey_hex)).hex()

# revocation_secret_hex = "4011111111111111111111111111111111111111111111111111111111111111"
# RL = hashlib.sha256(bytes.fromhex(revocation_secret_hex)).digest()
# revocation_lock_hex = RL.hex()

# merch_cpfp_privkey_hex = "2222222222222222222222222222222277777777777777777777777777777777"
# merch_cpfp_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_cpfp_privkey_hex)).hex()

# merch_fee_privkey_hex = "2222222222222222222222222222222266666666666666666666666666666666"
# merch_fee_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_fee_privkey_hex)).hex()

In [7]:
# # cpfp priv/pubkeys
# merch1_privkey_hex = "0221111111111111111111111111111100000000000000000000000000000000"
# merch1_privkey = bytes.fromhex(merch1_privkey_hex)
# merch1_pubkey = privkey_to_pubkey(merch1_privkey)

# merch2_privkey_hex = "2221111111111111111111111111111100000000000000000000000000000000"
# merch2_privkey = bytes.fromhex(merch2_privkey_hex)
# merch2_pubkey = privkey_to_pubkey(merch2_privkey).hex()

# merch3_privkey_hex = "3311111111111111111111111111111100000000000000000000000000000000"
# merch3_privkey = bytes.fromhex(merch3_privkey_hex)
# merch3_pubkey = privkey_to_pubkey(merch3_privkey).hex()

### Start up regtest mode
Delete any history so we are starting from scratch. <br>
Mine 101 blocks so we can spend some btc.

In [8]:
# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(2) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# start up bitcoind in regtest mode
# os.system("bitcoind -regtest -daemon -minrelaytxfee=0")
os.system("bitcoind -regtest -daemon")
time.sleep(2)

# generate 101 blocks so we can fund transactions
os.system("bitcoin-cli -regtest generate 101")
blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")

print("blockcount: " + str(blockcount))

blockcount: 101


Generate base58 address for the escrow funder

In [9]:
# Generate p2sh-p2wpkh address to fund the cust escrow funder
cust_privkey = bytes.fromhex(cust_funding_privkey_hex)
cust_public_key = privkey_to_pubkey(cust_privkey)
cust_p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(cust_public_key, testnet = True)

# print("Private key: " + privkey.hex())
# print("Public key: " + public_key.hex())
print("cust Address: " + cust_p2sh_p2wpkh_address)

# Generate p2sh-p2wpkh address to fund the cust escrow funder
merch_privkey = bytes.fromhex(merch_funding_privkey_hex)
merch_public_key = privkey_to_pubkey(merch_privkey)
merch_p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(merch_public_key, testnet = True)

# print("Private key: " + privkey.hex())
# print("Public key: " + public_key.hex())
print("merch Address: " + merch_p2sh_p2wpkh_address)

cust Address: 2MuuVDMJqfh2J9iS3DtfUeU1tEfff1SNahi
merch Address: 2MvoRVVCpkaZ9f6fVZZohxSMF69HfggsehJ


### Send btc to the escrow funder

In [10]:
cust_txid_1 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + cust_p2sh_p2wpkh_address + " 3.0")
print(cust_txid_1)

merch_txid_1 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + merch_p2sh_p2wpkh_address + " 4.0")
print(merch_txid_1)

8b3523da528a8f5a55c5374cfcebc661a0885e07a1b82a80c70c49febbd40bdd
310b64758ba9d11380714df4640f48b8ef3540570f70f9e1479cd84b42070b3e


In [12]:
# Find which output index the btc was sent to
cust_raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + cust_txid_1)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + cust_raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == cust_p2sh_p2wpkh_address:
    cust_index = 0
else:
    cust_index = 1
print("cust index: " + str(cust_index))

# Find which output index the btc was sent to
merch_raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_txid_1)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + merch_raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == merch_p2sh_p2wpkh_address:
    merch_index = 0
else:
    merch_index = 1
print("merch index: " + str(merch_index))

cust index: 1
merch index: 0


In [13]:
os.system("bitcoin-cli -regtest generate 1");

## Create Funding tx

In [16]:
raw_escrow_tx = subprocess.getoutput("python dual_funding_tx_with_changev2.py" 
                        + " --cust_txid " + cust_txid_1
                        + " --cust_index " + str(cust_index)
                        + " --cust_input_amount_btc " + "3.0"
                        + " --cust_funding_privkey " + cust_funding_privkey_hex
                        + " --merch_txid " + merch_txid_1
                        + " --merch_index " + str(merch_index)
                        + " --merch_input_amount_btc " + "4.0"
                        + " --merch_funding_privkey " + merch_funding_privkey_hex
                        + " --escrow_value_btc " + "2.1"
                        + " --cust_pubkey " + cust_pubkey_hex
                        + " --merch_pubkey " + merch_pubkey_hex
                        + " --cust_change_value_btc " + "0.89"
                        + " --cust_change_pubkey " + cust_change_pubkey_hex
                        + " --merch_change_value_btc " + "2.89"
                        + " --merch_change_pubkey " + merch_change_pubkey_hex)
print("serialized funding tx:\n" + raw_escrow_tx)

# Broadcast funding tx
escrow_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_escrow_tx + " true")
# "true" flag means we are okay with an 'absurdly' high tx fee

print("\nfunding txid:\n"+escrow_txid)

serialized funding tx:
02000000000102dd0bd4bbfe490cc7802ab8a1075e88a061c6ebfc4c37c5555a8f8a52da23358b0100000017160014578dd1183845e18d42f90b1a9f3a464675ad2440ffffffff3e0b07424bd89c47e1f9700f574035efb8480f64f44d718013d1a98b75640b31000000001716001403d761347fe4398f4ab398266a830ead7c9c2f30ffffffff038058840c00000000220020666c6bfa88ba97c90cb04c7038d56b5854e71a4dd174d79b1260c822a14f791e40084e0500000000160014fc7250a211deddc70ee5a2738de5f07817351cef40ca3911000000001600149f122ad09f6d53cf6838841560fccc6ca26ba28b024730440220023ae5a95db93c3aea4e5091fdb1b1d89ad8edd3d8475c34b258f3ad1b95e07e022036239c1f64bce694b660cf6ccb341d9bfce57b8d07c4d4f3436b9ff006dc885f0121021d5c5194a62b272f98a7d8321e4b3a03add9d61d13a15a00123888b81850cee202483045022100cbf4760798322f70f44902fbf21eef6432514c6f9a3a3dea7c27bc41ae585d330220034114c43fbc6760d21753553f47fea81b79a914f1507c1acbc21a4deded31dc012103780cd60a7ffeb777ec337e2c177e783625c4de907a4aee0f41269cc612fba45700000000

funding txid:
a8dbb9d38a7e2ab0b2cfa7bd0b0c2269eba69e484

In [17]:
os.system("bitcoin-cli -regtest generate 1");

#### Decode escrow funding transaction

In [18]:
# raw_escrow_tx1 = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + escrow_txid)
# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_escrow_tx1)
# print(decoded)

## Merch-close with CPFP

In [21]:
merch_close_cpfp_tx = subprocess.getoutput("python merch_close_with_cpfp.py"        
                + " --txid_str " + escrow_txid
                + " --index " + "0"
                + " --input_amount_btc " + "2.1"
                + " --cust_privkey " + cust_privkey_hex 
                + " --merch_privkey " + merch_privkey_hex
                + " --output_value_btc " + "1.99"
                + " --merch_payout_pubkey " + merch_payout_pubkey_hex   
                + " --to_self_delay " + "05cf" # number of blocks to delay to-self output   
                + " --merch_cpfp_pubkey " + merch_cpfp_pubkey_hex   
                + " --merch_cpfp_value_btc " + "0.1") # number of blocks to delay to-self output       
print("merch close with cpfp tx:\n" + merch_close_cpfp_tx)

NameError: name 'merch_cpfp_pubkey_hex' is not defined

This tx could be broadcast by itself and it would be a valid tx (cell below). If the fee was too small, we can effectively bump up the fee by creating a 'child' tx that spends from it

In [14]:
merch_close_cpfp_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + merch_close_cpfp_tx)
print(merch_close_cpfp_txid) 

9cef5178b6464d566532aad2510608a6c4c715490c7751b8d91387ba909905e6


In [15]:
os.system("bitcoin-cli -regtest generate 1");

In [16]:
mined_merch_close_cpfp_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_close_cpfp_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + mined_merch_close_cpfp_tx)
print(decoded)

{
  "txid": "9cef5178b6464d566532aad2510608a6c4c715490c7751b8d91387ba909905e6",
  "hash": "a6532fad966c8c52142a0fdab3ff5394284c3c6409190e0fb0a0d609d6fde768",
  "version": 2,
  "size": 347,
  "vsize": 181,
  "weight": 722,
  "locktime": 0,
  "vin": [
    {
      "txid": "bec3bf0c35dbdf678cf30d1293c7e1baf31851df7d8523cec302174c19119673",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "",
        "3045022100e61438b27c57dec59a89c172fa3a8c2bfc1fd761a0e79e7b0866c010ef7d9f81022054a6b8667867e6372c05d818cb8d7f9b10b6cc45b61d1e91e7912d04b4e0cc4501",
        "3045022100fa88798777fa9c6ba7e280128c8ad497fbf3045b5490ea12081f4c3b08d2877002200426167ebebbbd528c1d61b62e4533f5dbaf7f87bf52fe197f5e775ae99b236d01",
        "522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "valu

## Merch-fee input tx

fund another input that the merchant will use to add a large fee to the 'child' transaction.

In [17]:
# address to fund the merchant child tx
merch1_p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(merch1_pubkey, testnet = True)
# print("Address: " + merch1_p2sh_p2wpkh_address)

# Fund the merchant child input
txid_2 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + merch1_p2sh_p2wpkh_address + " 1.01")
print("txid:" + txid_2)

# Find which output index the btc was sent to
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_2)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == merch1_p2sh_p2wpkh_address:
    index = 0
else:
    index = 1
print("index: " + str(index))

txid:2afe95137e6e741d3267f96f8f1908baff88b796db030ce648384e378286acdd
index: 1


In [18]:
os.system("bitcoin-cli -regtest generate 1");

In [19]:
merch1_tx = subprocess.getoutput("python p2nsh_to_p2wpkh.py" 
                        + " --txid " + txid_2
                        + " --index " + str(index)
                        + " --input_amount_btc " + "1.01"
                        + " --input_privkey " + merch1_privkey_hex
                        + " --payout_value_btc " + "1"
                        + " --payout_pubkey " + merch2_pubkey)
# print(merch1_tx)

merch1_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + merch1_tx + " true")
# "true" flag means we are okay with absurdly high tx fee
print("merch fee outpoint:\n" + merch1_txid)

merch fee outpoint:
514c7ef5d1d94d6774753d78296a2b62b1b7f6e61cddbde6918bf7e243b536d9


## Create child transaction
Input[0]: Child outpoint <br>
Input[1]: Merch p2wpkh outpoint

Output[0]: Merch p2wpkh

In [20]:
merch_child_tx = subprocess.getoutput("python merch_close_child_tx.py" 
                        + " --child_txid_str " + merch_close_cpfp_txid
                        + " --child_index " + "1"
                        + " --child_privkey " + merch_cpfp_privkey_hex
                        + " --child_input_amount_btc " + "0.1"
                        + " --merch_txid_str " + merch1_txid
                        + " --merch_index " + "0"
                        + " --merch_privkey " + merch2_privkey_hex
                        + " --merch_input_amount_btc " + "1"
                        + " --output_value_btc " + "0.5"
                        + " --payout_pubkey " + merch3_pubkey)
print(merch_child_tx)

02000000000102e6059990ba8713d9b851770c4915c7c4a6080651d2aa3265564d46b67851ef9c0100000000ffffffffd936b543e2f78b91e6bddd1ce6f6b7b1622b6a29783d7574674dd9d1f57e4c510000000000ffffffff0180f0fa02000000001600140cbd8eef39d742140b81cf2f7fbade71af58a1820247304402203902507c229075f32eb6f9e2cc1c586219fcf993c12bc77724bf0492fd563043022024ff8a53a4d999f64793584aba73332db462b95e5eb35d7489dcd369129fa9120121025bdf02f6aeb3a5cbfdece604956a49571d823b8888f5c4d3ceb58e453c044e57024730440220335e18eff0525006a03e78be8657e3addf6e2cb0b7b53f98ed4db32a30dd5de902206941d961dd0491706f5a4bc82cd245da4251c4943c09afb5ce4371933ff80225012102c0947a47a59cb42316750ddd23d506d4c23ca997fbe40e9cb813970940501f4f00000000


#### Broadcast child tx

In [21]:
merch_child_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + merch_child_tx + " true")
print(merch_child_txid)

defc3281489701d40517c8a8119eb07e874dc5145edceeb0380a64a9cb667136


In [22]:
mined_merch_child_txid = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_child_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + mined_merch_child_txid)
print(decoded)

{
  "txid": "defc3281489701d40517c8a8119eb07e874dc5145edceeb0380a64a9cb667136",
  "hash": "c774120b46f3b9cf87b1be082434893f558e64dbabc1f77055a9169b0a4f6410",
  "version": 2,
  "size": 339,
  "vsize": 177,
  "weight": 708,
  "locktime": 0,
  "vin": [
    {
      "txid": "9cef5178b6464d566532aad2510608a6c4c715490c7751b8d91387ba909905e6",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "304402203902507c229075f32eb6f9e2cc1c586219fcf993c12bc77724bf0492fd563043022024ff8a53a4d999f64793584aba73332db462b95e5eb35d7489dcd369129fa91201",
        "025bdf02f6aeb3a5cbfdece604956a49571d823b8888f5c4d3ceb58e453c044e57"
      ],
      "sequence": 4294967295
    },
    {
      "txid": "514c7ef5d1d94d6774753d78296a2b62b1b7f6e61cddbde6918bf7e243b536d9",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "30440220335e18eff0525006a03e78be8657e3addf6e2cb0b7b53f98ed4db32a30dd5de9022